이번 튜토리얼에서는 

**로컬 드라이브에 저장**되어 있는 데이터셋을 로드하여 이미지 데이터셋을 구성하는 방법

에 대하여 알아보겠습니다.

**[참고]**
- `torchvision.transform`을 활용한 이미지 정규화는 [링크](https://teddylee777.github.io/pytorch/torchvision-transform)에서 확인해 주시기 바랍니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms

## 로컬 디렉토리에서 Image를 로드

튜토리얼 진행을 위한 데이터셋 다운로드
- `images` 폴더에 `rps` 데이터셋 다운로드

In [ ]:
# 이미지 데이터셋 다운로드
import urllib.request
import zipfile
import glob

url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
urllib.request.urlretrieve(url, 'rps.zip')
local_zip = 'rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('images/')
zip_ref.close()

print(glob.glob('images/rps/*'))

다운로드 받은 이미지는 `images`폴더 하위에 `rps` 폴더에 `rock`, `paper`, `scissors` 폴더가 생성되었으며, 각각의 폴더 밑에 가위/바위/보자기 사진 파일이 위치해 있습니다.

In [ ]:
# rock 폴더 하위에 위치한 .png 파일 10개 출력
glob.glob('images/rps/rock/*')[:10]

현재 `root` 디렉토리는 `images/rps` 폴더가 `root` 디렉토리입니다.

간단히 도식화 해보면 다음과 같습니다.

- `images/rps`
  - `rock` 폴더
  - `paper` 폴더
  - `scissor` 폴더

In [ ]:
image_folder = datasets.ImageFolder(root='images/rps', # 루트폴터 경로 지정
                                    transform=transforms.Compose([
                                        transforms.ToTensor(), 
                                    ])
                                   )

`image_folder` 변수에 `class_to_idx` 속성 값을 확인해보면 class에 맵핑되는 label을 확인할 수 있습니다.

In [ ]:
# class to index 라벨값 확인 (추후 시각화에 활용)
image_folder.class_to_idx

In [ ]:
train_loader = torch.utils.data.DataLoader(image_folder,         # image_folder를 지정
                                           batch_size=32,# 배치사이즈 지정
                                           shuffle=True,        # shuffle 여부 지정
                                           num_workers=8)       # num_workers 지정

In [ ]:
test_loader = torch.utils.data.DataLoader(image_folder,          # 원래는 별도의 Validation Set의 root 경로를 지정해야 한다. 
                                          batch_size=32, # 배치사이즈 지정
                                          shuffle=False,        # shuffle 여부 지정
                                          num_workers=8)         # num_workers 지정

1개의 배치(Batch)를 가져와서 shape를 확인하고, 시각화를 해봅니다.

In [ ]:
# 1개 batch 추출
images, labels = next(iter(train_loader))

In [ ]:
# images, labels에 각각 32개의 batch가 로드되어 있습니다.
# images는 300 X 300 사이즈 RGB 이미지 32장이 1개 batch로 구성되어 있습니다.
images.shape, labels.shape

`RPS` 데이터셋을 시각화 합니다.

In [ ]:
# ImageFolder의 속성 값인 class_to_idx를 할당
labels_map = {v:k for k, v in image_folder.class_to_idx.items()}

figure = plt.figure(figsize=(12, 8))
cols, rows = 8, 5

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(images), size=(1,)).item()
    img, label = images[sample_idx], labels[sample_idx].item()
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(torch.permute(img, (1, 2, 0)))
plt.show()